In [270]:
import pandas as pd
from dotenv import load_dotenv
import os
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build

load_dotenv()

gtm_id = os.environ["GTM_ACCOUNT_ID"]

In [271]:
def map_type(type):
    map = {
        '당사 쿠키': 'k',
        '상수': 'c',
        '데이터 영역 변수': 'v',
        'URL': 'u',
        '요소 속성': 'aev',
        '참고표': 'smm'
    }

    return map[type]
    

# Excel

In [291]:
xlsx_names = ['kyobo', 'library', 'scholar', 'story', 'tree']
service_names = {
    '교보문고_renewal': 'kyobo',
    '전자도서관': 'library',
    '스콜라': 'scholar',
    '스토리': 'story',
    '리딩트리': 'tree'
}

def get_xlsx():
    res = {}
    for name in service_names.values():
        res[name]=pd.ExcelFile(f'./xlsx/{name}.xlsx')

    return res

def get_sheet_name(excel):
    sheet_names = excel.sheet_names
    return sheet_names

def read_sheet(excel_file, sheet_name):
    return pd.read_excel(excel_file, sheet_name=sheet_name)

# GTM

In [292]:
# Get credentials
def get_credentials():
    SERVICE_ACCOUNT_FILE = './xlsx/gtm-api-409502-6f8362f190d7.json'

    credentials_container = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=["https://www.googleapis.com/auth/tagmanager.edit.containers"]
    )

    return credentials_container

credentials = get_credentials()

# Get service
def get_service():
    service = build('tagmanager', 'v2', credentials = credentials)

    return service

service = get_service()

# Get container
def get_container():
    parent = f"accounts/{gtm_id}"

    containers = service.accounts().containers().list(
        parent=parent,
    ).execute()

    return containers['container']

container_origin = get_container()
containers = {}
for i in container_origin:
    containers[i['name']] = i


# Set Container

In [300]:
service_name = '교보문고_renewal'
container = containers[service_name]

xlsxs = get_xlsx()
xlsx = xlsxs[service_names[service_name]]

# Get workspace
def get_workspaces():
    parent=container['path']

    workspace = service.accounts().containers().workspaces().list(
        parent=parent
    ).execute()

    return workspace['workspace']

workspace_origin = get_workspaces()

workspaces = {}
for i in workspace_origin:
    workspaces[i['name']] = i

workspace = workspaces['new']

In [301]:
def get_variables():
    parent=workspace['path']

    containers = service.accounts().containers().workspaces().variables().list(
        parent=parent
    ).execute()

    return containers

def create_gtm_variable(request_body):
    parent=workspace['path']

    try:
        response = service.accounts().containers().workspaces().variables().create(
            parent=parent,
            body=request_body
        ).execute()

        print(f"Created variable: {response['name']}")
        return response
    except Exception as e:
        print(f"An error occurred: {e}")
        print(f"Above error occurred with: {request_body['name']}")


In [302]:
def create_dataLayer_variable(sheet):
    error = 0
    for row in sheet.iloc():
        if (row[1] == '상수'):
            parameter = [{
                'type': 'template',
                'key': 'value',
                'value': row[2],
            }]
            type = 'c'
        elif (row[1] == '맞춤 자바스크립트'):
            parameter = [{
                'type': 'template',
                'key': 'javascript',
                'value': row[2],
            }]
            type = 'jsm'
        elif (row[1] == '당사 쿠키'):
            parameter = [{
                'type': 'template',
                'key': 'name',
                'value': row[2],
            }]
            type = 'k'
        else:
            parameter = [
                {
                    'type': 'template',
                    'key': 'name',
                    'value': row[2],
                },
                {
                    'type': 'integer',
                    'key': 'dataLayerVersion',
                    'value': 2,
                }
                ]
            type = 'v'

        request_body = {
            'name': row[0],
            'type': type,
            'parameter': parameter,
        }

        try:
            create_gtm_variable(request_body)
            time.sleep(5)
        except Exception as e:
            error += 1

    print(f"Error: {error}")


In [303]:
sheet_names = get_sheet_name(xlsx)

for i in sheet_names:
  print(i)

변수(당사쿠키)
변수(데이터 영역)
변수(URL)
변수(자동이벤트 변수)
변수(DOM 요소)
변수(맞춤JS)
변수(참고표)
변수(이벤트 매개변수)
트리거(맞춤 이벤트)
트리거(페이지뷰)
트리거(스크롤 깊이)
트리거(링크 클릭)(Web)
트리거(링크 클릭)(App)
트리거(모든 요소 클릭)(Web)
트리거(모든 요소 클릭)(App)
태그(맞춤HTML)
 태그(Web)
 태그(App 이벤트)


In [297]:
sheet = read_sheet(xlsx, sheet_names[0])
sheet1 = read_sheet(xlsx, sheet_names[1])
sheet2 = read_sheet(xlsx, sheet_names[5])

create_dataLayer_variable(sheet)
create_dataLayer_variable(sheet1)
create_dataLayer_variable(sheet2)

/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_20281/2634102849.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (row[1] == '상수'):
/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_20281/2634102849.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'value': row[2],
/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_20281/2634102849.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'name': row[0

Created variable: GA 측정 ID
Error: 0


/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_20281/2634102849.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif (row[1] == '맞춤 자바스크립트'):
/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_20281/2634102849.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif (row[1] == '당사 쿠키'):
/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_20281/2634102849.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[p

Created variable: DLV - category
Created variable: DLV - category2
Created variable: DLV - category3
Created variable: DLV - category4
Created variable: DLV - category5
Created variable: DLV - currency
Created variable: DLV - group_name
Created variable: DLV - group_user
Created variable: DLV - company_name
Created variable: DLV - readingtree_name
Created variable: DLV - readingtree_id
Created variable: DLV - reading_activation_score
Created variable: DLV - menu
Created variable: DLV - deposit
Created variable: DLV - read_point
Created variable: DLV - ecommerce
Created variable: DLV - ecommerce.deposit
Created variable: DLV - ecommerce.gift_card
Created variable: DLV - ecommerce.kyobo_cash
Created variable: DLV - ecommerce.section
Created variable: DLV - ecommerce.read_point
Created variable: DLV - ecommerce.total_point
Created variable: DLV - gift_card
Created variable: DLV - item_list_id
Created variable: DLV - item_list_name
Created variable: DLV - item_list_tab
Created variable: DL